In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torchvision import models
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import numpy as np
from collections import defaultdict

def train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(root=data_path, transform=transform)

    # Initialise K-Fold cross-validator
    kfold = KFold(n_splits=num_folds, shuffle=True)

    # Initialise containers for aggregated results
    all_preds = []
    all_labels = []

    # Start cross-validation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        # Sample elements randomly from a given list of indices for train and validation
        train_subsampler = Subset(dataset, train_ids)
        val_subsampler = Subset(dataset, val_ids)

        # Define data loaders for training and validation
        train_loader = DataLoader(dataset=train_subsampler, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_subsampler, batch_size=batch_size, shuffle=False)

        # Load a pre-trained AlexNet model
        model = models.alexnet(pretrained=True)

        # Modify the final layer to match the number of classes
        num_classes = len(dataset.classes)  # Automatically get the number of classes
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
        
        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        # Evaluation
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

    # Calculate and print the final aggregated classification report
    classes = dataset.classes  # Automatically get the class names from the dataset
    final_report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
    
    print('--------------------------------')
    print('Final Cross-Validation Results:')
    print(classification_report(all_labels, all_preds, target_names=classes))

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torchvision import models
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import numpy as np

def train_and_evaluate_AlexAdaGrad_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=8, num_folds=5):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(root=data_path, transform=transform)

    # Initialize K-Fold cross-validator
    kfold = KFold(n_splits=num_folds, shuffle=True)

    # Initialise containers for aggregated results
    all_preds = []
    all_labels = []

    # Start cross-validation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        # Sample elements randomly from a given list of indices for train and validation
        train_subsampler = Subset(dataset, train_ids)
        val_subsampler = Subset(dataset, val_ids)

        # Define data loaders for training and validation
        train_loader = DataLoader(dataset=train_subsampler, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_subsampler, batch_size=batch_size, shuffle=False)

        # Load a pre-trained AlexNet model
        model = models.alexnet(pretrained=True)

        # Modify the final layer to match the number of classes
        num_classes = len(dataset.classes)  # Automatically get the number of classes
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)  # Updated to AdaGrad optimizer

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        # Evaluation
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

    # Calculate and print the final aggregated classification report
    classes = dataset.classes  # Automatically get the class names from the dataset
    final_report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
    
    print('--------------------------------')
    print('Final Cross-Validation Results:')
    print(classification_report(all_labels, all_preds, target_names=classes))


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torchvision import models
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import numpy as np

def train_and_evaluate_AlexAdam_5fold(data_path, batch_size=16, learning_rate=0.0001, num_epochs=8, num_folds=5):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(root=data_path, transform=transform)

    # Initialize K-Fold cross-validator
    kfold = KFold(n_splits=num_folds, shuffle=True)

    # Initialize containers for aggregated results
    all_preds = []
    all_labels = []

    # Start cross-validation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')

        # Sample elements randomly from a given list of indices for train and validation
        train_subsampler = Subset(dataset, train_ids)
        val_subsampler = Subset(dataset, val_ids)

        # Define data loaders for training and validation
        train_loader = DataLoader(dataset=train_subsampler, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_subsampler, batch_size=batch_size, shuffle=False)

        # Load a pre-trained AlexNet model
        model = models.alexnet(pretrained=True)

        # Modify the final layer to match the number of classes
        num_classes = len(dataset.classes)  # Automatically get the number of classes
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Updated to Adam optimizer

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        # Evaluation
        model.eval()

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

    # Calculate and print the final aggregated classification report
    classes = dataset.classes  # Automatically get the class names from the dataset
    final_report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
    
    print('--------------------------------')
    print('Final Cross-Validation Results:')
    print(classification_report(all_labels, all_preds, target_names=classes))

In [4]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Viridis\Mel (Balanced)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.3486
Epoch [2/8], Loss: 0.7968
Epoch [3/8], Loss: 0.6561
Epoch [4/8], Loss: 0.4752
Epoch [5/8], Loss: 0.3916
Epoch [6/8], Loss: 0.3535
Epoch [7/8], Loss: 0.3254
Epoch [8/8], Loss: 0.2740
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.4120
Epoch [2/8], Loss: 0.8302
Epoch [3/8], Loss: 0.6503
Epoch [4/8], Loss: 0.5090
Epoch [5/8], Loss: 0.4090
Epoch [6/8], Loss: 0.3616
Epoch [7/8], Loss: 0.2927
Epoch [8/8], Loss: 0.2608
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.3981
Epoch [2/8], Loss: 0.9068
Epoch [3/8], Loss: 0.6598
Epoch [4/8], Loss: 0.5398
Epoch [5/8], Loss: 0.4477
Epoch [6/8], Loss: 0.3623
Epoch [7/8], Loss: 0.3301
Epoch [8/8], Loss: 0.3284
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.3934
Epoch [2/8], Loss: 0.8720
Epoch [3/8], Loss: 0.6619
Epoch [4/8], Loss: 0.5494
Epoch [5/8], Loss: 0.4467
Epoch [6/8], Loss: 0.3625
Epoch [7/8], Loss: 0.3229
Epoch [8/8], Loss: 0.2987
FOLD 5
--------------------------------
Epoch [1

In [5]:
train_and_evaluate_AlexAdaGrad_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.3716
Epoch [2/8], Loss: 0.7589
Epoch [3/8], Loss: 0.5179
Epoch [4/8], Loss: 0.3721
Epoch [5/8], Loss: 0.2651
Epoch [6/8], Loss: 0.2271
Epoch [7/8], Loss: 0.1808
Epoch [8/8], Loss: 0.1369
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.5008
Epoch [2/8], Loss: 0.8983
Epoch [3/8], Loss: 0.6204
Epoch [4/8], Loss: 0.4703
Epoch [5/8], Loss: 0.3658
Epoch [6/8], Loss: 0.2878
Epoch [7/8], Loss: 0.2286
Epoch [8/8], Loss: 0.1949
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.4042
Epoch [2/8], Loss: 0.7244
Epoch [3/8], Loss: 0.4859
Epoch [4/8], Loss: 0.3617
Epoch [5/8], Loss: 0.2558
Epoch [6/8], Loss: 0.2100
Epoch [7/8], Loss: 0.1447
Epoch [8/8], Loss: 0.1334
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.4602
Epoch [2/8], Loss: 0.8419
Epoch [3/8], Loss: 0.5124
Epoch [4/8], Loss: 0.3456
Epoch [5/8], Loss: 0.2514
Epoch [6/8], Loss: 0.1997
Epoch [7/8], Loss: 0.1442
Epoch [8/8], Loss: 0.1245
FOLD 5
--------------------------------
Epoch [1

In [6]:
train_and_evaluate_AlexAdam_5fold(data_path, batch_size=16, learning_rate=0.0001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.0818
Epoch [2/8], Loss: 0.5409
Epoch [3/8], Loss: 0.3754
Epoch [4/8], Loss: 0.2672
Epoch [5/8], Loss: 0.2364
Epoch [6/8], Loss: 0.2246
Epoch [7/8], Loss: 0.2067
Epoch [8/8], Loss: 0.1830
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.1479
Epoch [2/8], Loss: 0.5722
Epoch [3/8], Loss: 0.4221
Epoch [4/8], Loss: 0.2767
Epoch [5/8], Loss: 0.2514
Epoch [6/8], Loss: 0.2192
Epoch [7/8], Loss: 0.1878
Epoch [8/8], Loss: 0.1610
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.1608
Epoch [2/8], Loss: 0.5766
Epoch [3/8], Loss: 0.3744
Epoch [4/8], Loss: 0.3304
Epoch [5/8], Loss: 0.2516
Epoch [6/8], Loss: 0.2328
Epoch [7/8], Loss: 0.1983
Epoch [8/8], Loss: 0.1519
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.1613
Epoch [2/8], Loss: 0.5126
Epoch [3/8], Loss: 0.3731
Epoch [4/8], Loss: 0.2826
Epoch [5/8], Loss: 0.2568
Epoch [6/8], Loss: 0.1990
Epoch [7/8], Loss: 0.1627
Epoch [8/8], Loss: 0.1883
FOLD 5
--------------------------------
Epoch [1

In [7]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Viridis\Mel (Raw)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.6915
Epoch [2/8], Loss: 1.2310
Epoch [3/8], Loss: 1.0020
Epoch [4/8], Loss: 0.8533
Epoch [5/8], Loss: 0.7206
Epoch [6/8], Loss: 0.6202
Epoch [7/8], Loss: 0.5869
Epoch [8/8], Loss: 0.4885
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.6282
Epoch [2/8], Loss: 1.2029
Epoch [3/8], Loss: 0.9816
Epoch [4/8], Loss: 0.7851
Epoch [5/8], Loss: 0.7160
Epoch [6/8], Loss: 0.5912
Epoch [7/8], Loss: 0.5301
Epoch [8/8], Loss: 0.5298
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.6842
Epoch [2/8], Loss: 1.1767
Epoch [3/8], Loss: 0.9681
Epoch [4/8], Loss: 0.8318
Epoch [5/8], Loss: 0.6745
Epoch [6/8], Loss: 0.5706
Epoch [7/8], Loss: 0.4724
Epoch [8/8], Loss: 0.4109
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.7350
Epoch [2/8], Loss: 1.1657
Epoch [3/8], Loss: 0.9935
Epoch [4/8], Loss: 0.8189
Epoch [5/8], Loss: 0.7180
Epoch [6/8], Loss: 0.6000
Epoch [7/8], Loss: 0.5487
Epoch [8/8], Loss: 0.5014
FOLD 5
--------------------------------
Epoch [1

In [8]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Viridis\Mel (TSM)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.5565
Epoch [2/8], Loss: 1.0696
Epoch [3/8], Loss: 0.7827
Epoch [4/8], Loss: 0.6351
Epoch [5/8], Loss: 0.5398
Epoch [6/8], Loss: 0.4567
Epoch [7/8], Loss: 0.3799
Epoch [8/8], Loss: 0.3115
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.5065
Epoch [2/8], Loss: 1.0558
Epoch [3/8], Loss: 0.8311
Epoch [4/8], Loss: 0.6602
Epoch [5/8], Loss: 0.5768
Epoch [6/8], Loss: 0.4503
Epoch [7/8], Loss: 0.4335
Epoch [8/8], Loss: 0.3558
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.5193
Epoch [2/8], Loss: 1.0247
Epoch [3/8], Loss: 0.7668
Epoch [4/8], Loss: 0.6081
Epoch [5/8], Loss: 0.5034
Epoch [6/8], Loss: 0.4181
Epoch [7/8], Loss: 0.3734
Epoch [8/8], Loss: 0.3761
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.4798
Epoch [2/8], Loss: 0.9463
Epoch [3/8], Loss: 0.7842
Epoch [4/8], Loss: 0.6060
Epoch [5/8], Loss: 0.5274
Epoch [6/8], Loss: 0.4034
Epoch [7/8], Loss: 0.3858
Epoch [8/8], Loss: 0.3172
FOLD 5
--------------------------------
Epoch [1

In [9]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Viridis\Mel (SSiA)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.3632
Epoch [2/8], Loss: 0.8871
Epoch [3/8], Loss: 0.7206
Epoch [4/8], Loss: 0.5995
Epoch [5/8], Loss: 0.4946
Epoch [6/8], Loss: 0.3957
Epoch [7/8], Loss: 0.3630
Epoch [8/8], Loss: 0.3347
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.3866
Epoch [2/8], Loss: 0.9245
Epoch [3/8], Loss: 0.6734
Epoch [4/8], Loss: 0.5369
Epoch [5/8], Loss: 0.4408
Epoch [6/8], Loss: 0.3749
Epoch [7/8], Loss: 0.3529
Epoch [8/8], Loss: 0.3480
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.3895
Epoch [2/8], Loss: 0.8496
Epoch [3/8], Loss: 0.6549
Epoch [4/8], Loss: 0.5396
Epoch [5/8], Loss: 0.4570
Epoch [6/8], Loss: 0.3819
Epoch [7/8], Loss: 0.3568
Epoch [8/8], Loss: 0.3139
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.4451
Epoch [2/8], Loss: 0.9289
Epoch [3/8], Loss: 0.7139
Epoch [4/8], Loss: 0.6082
Epoch [5/8], Loss: 0.4697
Epoch [6/8], Loss: 0.4014
Epoch [7/8], Loss: 0.3856
Epoch [8/8], Loss: 0.3277
FOLD 5
--------------------------------
Epoch [1

In [10]:
data_path = r"C:\Users\44778\OneDrive\Desktop\UWE_Docs\7. Dissertation\CNN Data Files\Old_Data\Viridis\Mel (TSM + SSiA)"
train_and_evaluate_AlexSGD_5fold(data_path, batch_size=16, learning_rate=0.01, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.3051
Epoch [2/8], Loss: 0.7810
Epoch [3/8], Loss: 0.5672
Epoch [4/8], Loss: 0.4413
Epoch [5/8], Loss: 0.3901
Epoch [6/8], Loss: 0.3170
Epoch [7/8], Loss: 0.2830
Epoch [8/8], Loss: 0.2463
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.2479
Epoch [2/8], Loss: 0.7228
Epoch [3/8], Loss: 0.5531
Epoch [4/8], Loss: 0.4379
Epoch [5/8], Loss: 0.3682
Epoch [6/8], Loss: 0.3045
Epoch [7/8], Loss: 0.2713
Epoch [8/8], Loss: 0.2443
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.2506
Epoch [2/8], Loss: 0.7227
Epoch [3/8], Loss: 0.5491
Epoch [4/8], Loss: 0.4185
Epoch [5/8], Loss: 0.3560
Epoch [6/8], Loss: 0.3052
Epoch [7/8], Loss: 0.2607
Epoch [8/8], Loss: 0.2469
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.2727
Epoch [2/8], Loss: 0.7399
Epoch [3/8], Loss: 0.5459
Epoch [4/8], Loss: 0.4404
Epoch [5/8], Loss: 0.3774
Epoch [6/8], Loss: 0.3165
Epoch [7/8], Loss: 0.2722
Epoch [8/8], Loss: 0.2644
FOLD 5
--------------------------------
Epoch [1

In [11]:
train_and_evaluate_AlexAdaGrad_5fold(data_path, batch_size=16, learning_rate=0.001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.0881
Epoch [2/8], Loss: 0.4682
Epoch [3/8], Loss: 0.2887
Epoch [4/8], Loss: 0.1956
Epoch [5/8], Loss: 0.1475
Epoch [6/8], Loss: 0.1111
Epoch [7/8], Loss: 0.0922
Epoch [8/8], Loss: 0.0718
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.2328
Epoch [2/8], Loss: 0.5983
Epoch [3/8], Loss: 0.3728
Epoch [4/8], Loss: 0.2645
Epoch [5/8], Loss: 0.1924
Epoch [6/8], Loss: 0.1524
Epoch [7/8], Loss: 0.1125
Epoch [8/8], Loss: 0.0919
FOLD 3
--------------------------------
Epoch [1/8], Loss: 1.2999
Epoch [2/8], Loss: 0.5693
Epoch [3/8], Loss: 0.3421
Epoch [4/8], Loss: 0.2270
Epoch [5/8], Loss: 0.1682
Epoch [6/8], Loss: 0.1314
Epoch [7/8], Loss: 0.1000
Epoch [8/8], Loss: 0.0757
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.2690
Epoch [2/8], Loss: 0.6021
Epoch [3/8], Loss: 0.3737
Epoch [4/8], Loss: 0.2534
Epoch [5/8], Loss: 0.1983
Epoch [6/8], Loss: 0.1437
Epoch [7/8], Loss: 0.1338
Epoch [8/8], Loss: 0.1066
FOLD 5
--------------------------------
Epoch [1

In [12]:
train_and_evaluate_AlexAdam_5fold(data_path, batch_size=16, learning_rate=0.0001, num_epochs=8, num_folds=5)

FOLD 1
--------------------------------


c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\44778\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/8], Loss: 1.0951
Epoch [2/8], Loss: 0.5032
Epoch [3/8], Loss: 0.3356
Epoch [4/8], Loss: 0.2684
Epoch [5/8], Loss: 0.2095
Epoch [6/8], Loss: 0.1668
Epoch [7/8], Loss: 0.1593
Epoch [8/8], Loss: 0.1122
FOLD 2
--------------------------------
Epoch [1/8], Loss: 1.0161
Epoch [2/8], Loss: 0.5122
Epoch [3/8], Loss: 0.3304
Epoch [4/8], Loss: 0.2677
Epoch [5/8], Loss: 0.2361
Epoch [6/8], Loss: 0.1765
Epoch [7/8], Loss: 0.1725
Epoch [8/8], Loss: 0.1509
FOLD 3
--------------------------------
Epoch [1/8], Loss: 0.9841
Epoch [2/8], Loss: 0.4926
Epoch [3/8], Loss: 0.3369
Epoch [4/8], Loss: 0.2514
Epoch [5/8], Loss: 0.2221
Epoch [6/8], Loss: 0.1650
Epoch [7/8], Loss: 0.1665
Epoch [8/8], Loss: 0.1338
FOLD 4
--------------------------------
Epoch [1/8], Loss: 1.0268
Epoch [2/8], Loss: 0.4965
Epoch [3/8], Loss: 0.3326
Epoch [4/8], Loss: 0.2565
Epoch [5/8], Loss: 0.2302
Epoch [6/8], Loss: 0.1855
Epoch [7/8], Loss: 0.1417
Epoch [8/8], Loss: 0.1482
FOLD 5
--------------------------------
Epoch [1

: 